# Loading Farms to Freeways from the API and ro-crate metadata file

The Language Data Commons of Australia (LDaCa) packages all their data collections in an [ro-crate](https://www.researchobject.org/ro-crate/). There is a metadata file called `ro-crate-metadata.json` that comes with every data collection and this is how we can obtain metadata on this collection of research objects.

The metadata file is in the json format, and so we'll be learning how to read a json file in this notebook.

<div class="alert alert-block alert-success">
<b>Skills</b> 
    
<ul>
<li> json file format (see https://en.wikipedia.org/wiki/JSON)</li>
<li> working with dataframes, via pandas</li>
<li> discovering and exploring metadata</li>
<li> extracting ngrams, via textacy</li>
</ul>    
<br>

<b>Skill level:</b> Intermediate
</div>

This notebook uses the library 'requests', as shown in the [Using APIs: Open Australia](https://github.com/Australian-Text-Analytics-Platform/open-australia-api/blob/main/api.ipynb) notebook. If you haven't already familiarised yourself with that notebook, it might be a good idea to do so first.

In [ ]:
%%capture
# Before we begin, let's make sure that we install all the requirements that we need
import sys
!{sys.executable} -m pip install -r requirements.txt

## Import libraries

Python needs the libraries that will be used by the notebook to be specified before they are used. We do this with the reserved word `import`, as shown below.

In [ ]:
import json                       # json library to read json file formats
import pprint                     # Prints in a nice way
import requests                   # Uses the requests library for REST apis
import os                         # Loads operating system libraries
from ldaca.ldaca import LDaCA     # Loads the LDaCA ReST api wrapper
from rocrate_lang.utils import as_list # A handy utility for converting to list

## Variables 

We need to specify the path to the data collection. This `Farms to Freeways` data collection was used with permission by [The University of Western Sydney](https://omeka.uws.edu.au/farmstofreeways/). It was made into an ro-crate by the [LDaCA](https://ardc.edu.au/project/language-data-commons-of-australia-ldaca/) project and it is the data set used here to demonstrate the skills list above.

The variables below refer to the `path` where the collection can be found. There are also variables below that refer to ro-crates as specified in LDaCA profiles, for example all artefacts of importances are called `RepositoryObject`, and when an artefact is linked to, it is done with a `hasFile` keyword in the ro-crate metadata file.
<div class="alert alert-block alert-success">
Create a file, name it vars.env and store your API_KEY. This will be required for downloading any files. Go to the <a href='https://data.atap.edu.au/'>ATAP LDaCA Portal</a> and generate an API Key

Example vars.env:

API_KEY=12345

</div>

In [ ]:
# Specify location where collection is
LDACA_API = 'https://data.atap.edu.au/api'
COLLECTION_ID = 'arcp://name,farms-to-freeways-example-dataset'
from dotenv import load_dotenv    # loads environment variables
load_dotenv('vars.env') # load the environment variables located in the vars.env files
API_TOKEN = os.getenv('API_KEY') # store your environment variable in this jupyter notebook
if not API_TOKEN:
    print("Set a variable in the vars.env file and name API_KEY")

In [ ]:
# Get the Farms to Freeways ro-crate metadata by passing the arcpId in as a parameter to the get request

ldaca = LDaCA(url=LDACA_API, token=API_TOKEN, data_dir='data')
ldaca.retrieve_collection(collection=COLLECTION_ID, collection_type='Collection', data_dir='data')

metadata = ldaca.crate

# Inspect the metadata (Currently commented out for brevity)
metadata

### ro-crate Profiles

An ro-crate profile is a set of conventions that tell us what elements an ro-crate minimally contains.

These profiles tell us what to expect to find in the data packages. Learn more about them here: https://www.researchobject.org/ro-crate/profiles.html

In [ ]:
# TYPE values should be lists. 
# We define a PRIMARY_OBJECT as a 'RepositoryObject' because that is where the main data is stored 
PRIMARY_OBJECT = 'RepositoryObject'

### Define Variables to Gather Metadata

As suggested above, there are '@types' that define certain objects within the collection, for example there is type called 'Person'. This json object stores information such as 'birthDate' about the 'Person'. In the code block below, we discover all the types stored in this metadata file. 

In [ ]:
# Find all types and find types that have linked objects
files = set()
types = list()
primary_object_types = list()

# Lets see what we can find in our metadata
for entity in ldaca.crate.contextual_entities + ldaca.crate.data_entities:
    entity_type = as_list(entity.type)  # We make sure that each type is a list
    for e_t in entity_type:
        types.append(e_t)


## Exploring the Metadata

Anytime you work with data, it's always a good idea to inspect it by printing it out.

In [ ]:
# Print the variables
# All the types, removing duplicates
list(dict.fromkeys(types))

## Primary Objects

The primary object types are the ones we may care about, so we will pull them into their own dataframe:

We use the special function called `ldaca.crate.dereference(id)` to find out which linked object this is
More on how to consume an ro-crate using python here: https://github.com/ResearchObject/ro-crate-py#consuming-an-ro-crate

In [ ]:
# Types of PRIMARY_OBJECTs ie [PRIMARY_OBJECT, X]. What kinds of Xs do we have?
for entity in ldaca.crate.contextual_entities + ldaca.crate.data_entities:
    if 'RepositoryObject' in as_list(entity.type):
        print(entity.get('name'))
        item = ldaca.crate.dereference(entity.id)
        primary_object_types.append(item.as_jsonld())

<div class="alert alert-block alert-info">
<b>Python Library: pandas (dataframe)</b> 
<br>    
A dataframe is akin to a table -- it is made up of rows and columns. 
<br>    
In the block of code below, we are creating a dataframe for each "primary_object_type" ('Person', 'TextDialogue', 'Photographic image', and 'Text')
</div>

In [ ]:
import pandas as pd  # this means we will refer to pandas as 'pd' throughout the code

primary_objects_dataframe = pd.json_normalize(primary_object_types)
primary_objects_dataframe

## hasPart

Each RepositoryObject has a list of files in an array called `hasPart`

In [ ]:
# Types of File that are in each primary object. What kinds of files do we have?
for entity in primary_object_types:
    if 'hasPart' in entity:
        hasPart = entity.get('hasPart')
        for part in as_list(hasPart):
            file = ldaca.crate.dereference(part.get('@id'))
            files.add(file)
print(f"{len(files)} files")

## Files

We extracted the files from the primary objects we cared about, now lets filter the CSVs by searching each file of the type `Annotation` and then making sure we get a csv file by testing the `encodingFormat`

In [ ]:

annotations = set()
csvs = list()

# pick out the annotation files
for file in files:
    if 'Annotation' in as_list(file.type):
        annotations.add(file)

# from this annotations select only the CSVs

for annotation in annotations:
    if annotation.get('encodingFormat') == 'text/csv':
        annotation_json = annotation.as_jsonld()
        csvs.append(annotation_json)
    
 
print(f"We have {len(csvs)} csv objects")   


## CSVs

Lets explore the metadata of one csv object

In [ ]:
csv = next(iter(csvs))
print(json.dumps(csv, indent=2, sort_keys=False))

## Normalize CSV objects

Lets use pandas json_normalize to create a data frame for speakers

In [ ]:
csvs_dataframe = pd.json_normalize(csvs)
    
list(csvs_dataframe)

In [ ]:
csvs_dataframe.iloc[0]['@id']

## Speakers

Each RepositoryObject has speakers. Lets find them in a similar way we did with the files

In [ ]:
speakers = list()

for entity in primary_object_types:
    if 'speaker' in entity:
        speaker = entity.get('speaker')
        if speaker:
            for person in as_list(speaker):
                speaker_item = ldaca.crate.dereference(person['@id'])
                speakers.append(speaker_item.as_jsonld())
print(f"{len(speakers)} speakers")

## Person
Each speaker is represented by a `Person` object lets explore one

In [ ]:
person = next(iter(speakers))
print(json.dumps(person, indent=2, sort_keys=False))

## Normalize Speakers

Lets use pandas json_normalize to create a data frame for speakers

In [ ]:
speakers_dataframe = pd.json_normalize(speakers)
    
speakers_dataframe   

In [ ]:
#Lets use 'indexableText.@id' to join the csvs in the primary objects dataframe

primary_objects_dataframe.iloc[195]['indexableText.@id']

In [ ]:
df_1 = pd.merge(left=csvs_dataframe, right=primary_objects_dataframe, left_on='@id', right_on="indexableText.@id",
              suffixes=('_csvs', '_po'), how='left')
df_1

In [ ]:
# Now lets use speaker@id to join with the dataframe we have just created
df_1.iloc[0]['speaker.@id']

In [ ]:
new = pd.merge(left=speakers_dataframe, right=df_1, left_on='@id', right_on="speaker.@id",
              suffixes=('_speaker', '_artefact'), how='left')
new

## Statistical Summaries

In the metadata there is a key called "birthDate" which is a string that only has the birth year of the speaker. One of the birthDate values in the metadata has a string value "c 1924", instead of a simply sequence of digits there is, as shown when the list of birthDates are printed

### Birth Year

In [ ]:
new.birthDate_speaker

If 'birthDate' only has the year listed as a text string (str), we would need to convert the birthDate value from str to an integer (int) if we want to do any statistical operations based on birthDate. This conversion can be done by 'type casting', eg, if year is a string that has the value "1924", we simply impose the type int(), such as int(year), so the string year = "1924" => integer, year = 1924, which is then a number (not a string) that can undergo maths operations.

The function int(...) in the following line imposes an integer type conversion. That is, int(x) converts x from whatever type it is into an integer, as long as it makes sense for x to be converted into an integer. For example if x = "abc", then it would be impossible to know what value x would have as an integer. But if a string x = "1924", then the integer would have the value 1924.

In [ ]:
# Normalising the birth year and casting them as integers

new.birthDate_speaker = new.birthDate_speaker.apply(lambda year: year if (type(year) == int) else int(year[-4:]))
new.birthDate_speaker

<div class="alert alert-block alert-info">
<b>Python Library: datetime</b> 

<br>    
    
The library datetime can provide the current date and time and allows us to do calculations over any date and time, such as determining the difference between time zones.
<br>    
</div>

In [ ]:
# Import the module
import datetime

# We can calculate the mean (average) age
this_year = datetime.datetime.now().year
# Create a list called 'age' which takes every year in birth_year as y. Then get this_year and minus that
# number from year y and make sure all those numbers are stored in a list, which is why we have [] around the
# whole sequence of instructions below.
age = [this_year - y for y in new.birthDate_speaker]

# Print the list of the age of all the speakers if they were all alive today
age

<div class="alert alert-block alert-info">
<b>Python Library: statistics</b> 

<br>    
    
The statistics library provides functions to calculate simple statistics, such as the mean, mode, standard deviation, etc., over numeric data.<br>    
</div>

In [ ]:
# Import the module
import statistics

print('== AGE ==')
# Print the mean age, which is the average age of all the speakers
print('MEAN:', statistics.mean(age))
# The mode is the most freqently occur age. That is, there are more speakers of this age than any other.
print('MODE:', statistics.mode(age))
# The median is the middle value if the age of the participants were listed in order.
print('MEDIAN:', statistics.median(age))
# The standard deviation is a statistical metric that gives us an indication of how dispersed the age range of the speakers is
print('STD DEV:', "{:.1f}".format(statistics.stdev(age)))
print()

print('== BIRTH YEAR ==')
# Print the mean, median, mode and standard deviation of the birth year
print('MEAN:', statistics.mean(new.birthDate_speaker))
print('MEDIAN:', statistics.median(new.birthDate_speaker))
print('MODE:', statistics.mode(new.birthDate_speaker))

<div class="alert alert-block alert-info">
<b>The Counter Container</b> 

<br>    
    
The Counter container monitors the number of equivalent elements that have been added to it. Learn more about it here: https://docs.python.org/3/library/collections.html#collections.Counter<br>    
</div>

### Other Metadata Features: Place

There are other metadata columns that require normalising in this dataframe. For example there is a location 'Penrith' as well as 'Kingston, Penrith', and there is a location 'St. Marys' as well as 'St Marys' (no '.'), as shown when you print the 'address' column.

In [ ]:
new.address_speaker

In [ ]:
# Let's normalise these locations within the dataframe 'new'
# NOTE 'address' is the where the story told in the interview takes place
new.address_speaker = new['address_speaker'].apply(lambda place: place.split(',')[-1].replace('.', '').strip())
new.address_speaker

In [ ]:
from collections import Counter

place_of_story = new.address_speaker
place_of_birth = new.birthDate_speaker

In [ ]:
# How many of the interviews talked about a certain location/city/suburb
count_story_place = dict(Counter(new.address_speaker))
pprint.pprint(count_story_place)

In [ ]:
# Count place of birth
count_birth_place = dict(Counter(new.birthDate_speaker))
pprint.pprint(count_birth_place)

### Cross-cutting 2 features found in the metadata



In [ ]:
# Let's try with 1 suburb, where the suburb = Blacktown
suburb = new.loc[new['address_speaker'] == 'Penrith']
# For all stories set in this suburb, print all the storytellers' birth year 
print(suburb.birthDate_speaker)

In [ ]:
# Print place of story and speaker's birth year
all_suburbs = list(count_story_place.keys())
all_suburbs

In [ ]:
names = list(suburb.name)
names = list( dict.fromkeys(names) )

In [ ]:
names

In [ ]:
# Traverse through the suburbs and print the data we are interested in
# In addition, let's save this information in a dictionary called 'suburbs'
# so we don't have to bother with dataframes
suburbs = dict()
for s in all_suburbs:
    suburbs[s] = dict()
    places = new.loc[(new['address_speaker'] == s)]
    print('## ===', s, '-- total:', len(places))
    # NOTE: index is the internal reference for the row in the dataframe called 'p'
    for index, i in places.iterrows():
        # initialise each person's info
        person = dict()
        # name
        name = i['name']
        print(name)
        # birthPlace
        birthPlace = i['birthPlace_speaker']
        person['birthPlace_speaker'] = birthPlace
        print(birthPlace)
        # birthDate
        birthDate = i['birthDate_speaker']
        person['birthDate_speaker'] = birthDate
        print(birthDate)        
        # dialogue files
        person_files = []
        print(person)
        if i['indexableText.@id']:
            person_files.append(i['indexableText.@id'])
#             if f['@id']:
#                 print(f['@id'])
#                 if f['@id'].endswith('.csv'):
#                     print(f['@id'])
#                     person_files.append(f['@id'])
        person['files'] = person_files
        print(person_files)
        print()
        
        suburbs[s].update({name: person})

In [ ]:
pprint.pprint(suburbs)

### Sanity Check

Let's print out the information on 1 person to check that our data is looking as we expect it to.

In [ ]:
NAME = 'Patricia Parker'
PLACE = 'Blacktown'

# Print the whole dict structure for Amelia Vincent
suburbs[PLACE][NAME]

In [ ]:
# Print the birthPlace
suburbs[PLACE][NAME]['birthPlace_speaker']

<div class="alert alert-block alert-info">
    <b>Downloading a file from the ReST API</b> 
    <br>    
    We have the reference of each file stored in LDaCA. 
    <br>    
    We use pandas to download and attach the API_TOKEN to each request
</div>

In [ ]:
# files are a list, so let's create a list of dataframes to save all the contents of each file
dataframes = list()  # we have a list of files so let's save them as a list of dataframes
for f in suburbs[PLACE][NAME]['files']:
    df = pd.read_csv(f, storage_options={'Authorization': 'Bearer %s' % API_TOKEN})
    df.fillna('', inplace=True)
    dataframes.append(df)

# How many files are there in the list?
len(dataframes)

In [ ]:
# There is only 1 file in the list. Actually there is only ever 1 file in every files list
dataframes[0]

## Counting BiGrams

Let's use textacy and spacy to process the text from each of the files and count the bigrams.

<div class="alert alert-block alert-info">
<b>Text Processing</b> 
<br>    
<ul>
    <li>textacy: to find bigrams</li>
    <li>spacy: to ingest and process the text</li>
</ul>    
    
<br>    
</div>

In [ ]:
import textacy
import spacy

# Load the language model
nlp = spacy.load("en_core_web_sm")

In [ ]:
blacktown = list()
print('## == BLACKTOWN')
for person in suburbs['Blacktown']:
    person_data = suburbs['Blacktown'][person]
    file = person_data['files'][0]
    df = pd.read_csv(file, storage_options={'Authorization': 'Bearer %s' % API_TOKEN})
    df.fillna('', inplace=True)
    text = list(df.text)
    blacktown.extend(text)
    print(person)
    print('\tCUMULATIVE TOTAL', len(blacktown))

In [ ]:
penrith = list()
print('## == Penrith')
for person in suburbs['Penrith']:
    person_data = suburbs['Penrith'][person]
    file = person_data['files'][0]
    df = pd.read_csv(file, storage_options={'Authorization': 'Bearer %s' % API_TOKEN})
    df.fillna('', inplace=True)
    text = list(df.text)
    penrith.extend(text)
    print(person)
    print('\tCUMULATIVE TOTAL', len(penrith))

In [ ]:
text_b = nlp(' '.join(blacktown))
ngrams_b = list(textacy.extract.basics.ngrams(text_b, 2, min_freq=10))

In [ ]:
words_b = [w.text.lower() for w in ngrams_b]

In [ ]:
from collections import Counter
cb = Counter(words_b)
cb

In [ ]:
text_p = nlp(' '.join(penrith))
ngrams_p = list(textacy.extract.basics.ngrams(text_p, 2, min_freq=10))
words_p = [w.text.lower() for w in ngrams_p]
cp = Counter(words_p)
cp

In [ ]:
overlapping = [w for w in cp if w in cb]
overlapping

In [ ]:
unique_blacktown = [w for w in cb if w not in cp]
unique_blacktown

In [ ]:
unique_penrith = [w for w in cp if w not in cb]
unique_penrith

In [ ]:
birth_year_blacktown = [suburbs['Blacktown'][name]['birthDate_speaker'] for name in suburbs['Blacktown']]
birth_year_blacktown.sort()
birth_year_blacktown

In [ ]:
birth_year_penrith = [suburbs['Penrith'][name]['birthDate_speaker'] for name in suburbs['Penrith']]
birth_year_penrith.sort()
birth_year_penrith

### Counting n-grams for Suburb given Birth Year

In [ ]:
blacktown_pre20s = list()
print('## == BLACKTOWN PRE-1920')
for person in suburbs['Blacktown']:
    person_data = suburbs['Blacktown'][person]
    if person_data['birthDate_speaker'] < 1920:
        file = person_data['files'][0]
        df = pd.read_csv(file, storage_options={'Authorization': 'Bearer %s' % API_TOKEN})
        df.fillna('', inplace=True)
        text = list(df.text)
        blacktown_pre20s.extend(text)
        print(person)
        print('\tCUMULATIVE TOTAL', len(blacktown_pre20s))

In [ ]:
penrith_pre20s = list()
print('## == PENRITH PRE-1920')
for person in suburbs['Penrith']:
    person_data = suburbs['Penrith'][person]
    if person_data['birthDate_speaker'] < 1920:
        file = person_data['files'][0]
        df = pd.read_csv(file, storage_options={'Authorization': 'Bearer %s' % API_TOKEN})
        df.fillna('', inplace=True)
        text = list(df.text)
        penrith_pre20s.extend(text)
        print(person)
        print('\tCUMULATIVE TOTAL', len(penrith_pre20s))

In [ ]:
text_b20 = nlp(' '.join(blacktown_pre20s))
ngrams_b20 = list(textacy.extract.basics.ngrams(text_b20, 2, min_freq=10))
words_b20 = [w.text.lower() for w in ngrams_b20]
cb20 = Counter(words_b20)
cb20

In [ ]:
text_p20 = nlp(' '.join(penrith_pre20s))
ngrams_p20 = list(textacy.extract.basics.ngrams(text_p20, 2, min_freq=10))
words_p20 = [w.text.lower() for w in ngrams_p20]
cp20 = Counter(words_p20)
cp20

<div class="alert alert-block alert-success">
<b>You Can Extend this Notebook</b> 
    
<ul>
<li> You change this notebook studying different suburbs.</li>
<li> Rather than examining the vocabulary of those born before 1920, you can look at the stories of those who were born later.</li>
<li> Try looking at unigrams or trigrams instead of bigrams.</li>
<li> The minimum frequency of bigrams was 10. You can increase or decrease this threshold.</li>
</ul>    
<br>
</div>